# Question 2

# Architecture 1: Simple CNN

In [2]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(8*8*16, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(-1, 8*8*16)
        x = self.fc(x)
        return x


# Architecture 2: LeNet-5

In [3]:
import torch
import torch.nn as nn

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Architecture 3: MiniVGGNet

In [4]:
import torch
import torch.nn as nn

class MiniVGGNet(nn.Module):
    def __init__(self):
        super(MiniVGGNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*8*8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.pool(x)
        x = x.view(-1, 64*8*8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Architecture 4: MobileNetV1

In [5]:
import torch
import torch.nn as nn

class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class MobileNetV1(nn.Module):
    def __init__(self):
        super(MobileNetV1, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.dw_separable1 = DepthwiseSeparableConv(32, 64, stride=1)
        self.dw_separable2 = DepthwiseSeparableConv(64, 128, stride=2)
        self.dw_separable3 = DepthwiseSeparableConv(128, 128, stride=1)
        self.dw_separable4 = DepthwiseSeparableConv(128, 256, stride=2)
        self.dw_separable5 = DepthwiseSeparableConv(256, 256, stride=1)
        self.dw_separable6 = DepthwiseSeparableConv(256, 512, stride=2)
        self.dw_separable7 = DepthwiseSeparableConv(512, 512, stride=1)
        self.dw_separable8 = DepthwiseSeparableConv(512, 1024, stride=2)
        self.dw_separable9 = DepthwiseSeparableConv(1024, 1024, stride=1)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.dw_separable1(x)
        x = self.dw_separable2(x)
        x = self.dw_separable3(x)
        x = self.dw_separable4(x)
        x = self.dw_separable5(x)
        x = self.dw_separable6(x)
        x = self.dw_separable7(x)
        x = self.dw_separable8(x)
        x = self.dw_separable9(x)
        x = self.avg_pool(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x


# Architecture 5: ShuffleNetV1

In [6]:
import torch
import torch.nn as nn

class ChannelShuffle(nn.Module):
    def __init__(self, groups):
        super(ChannelShuffle, self).__init__()
        self.groups = groups

    def forward(self, x):
        batch_size, num_channels, height, width = x.size()
        channels_per_group = num_channels // self.groups

        # Reshape
        x = x.view(batch_size, self.groups, channels_per_group, height, width)

        # Transpose
        x = x.transpose(1, 2).contiguous()

        # Reshape back
        x = x.view(batch_size, -1, height, width)
        return x

class ShuffleNetV1(nn.Module):
    def __init__(self, groups=3):
        super(ShuffleNetV1, self).__init__()
        self.conv1 = nn.Conv2d(3, 24, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.stage2 = self._make_stage(24, 144, groups)
        self.stage3 = self._make_stage(144, 288, groups)
        self.stage4 = self._make_stage(288, 576, groups)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(576, 10)

    def _make_stage(self, in_channels, out_channels, groups):
        layers = []
        layers.append(nn.Conv2d(in_channels, out_channels // 2, kernel_size=1))
        layers.append(nn.BatchNorm2d(out_channels // 2))
        layers.append(self.relu)
        layers.append(ChannelShuffle(groups))
        layers.append(nn.Conv2d(out_channels // 2, out_channels // 2, kernel_size=3, stride=1, padding=1, groups=out_channels // 2))
        layers.append(nn.BatchNorm2d(out_channels // 2))
        layers.append(nn.Conv2d(out_channels // 2, out_channels, kernel_size=1))
        layers.append(nn.BatchNorm2d(out_channels))
        layers.append(self.relu)
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.max_pool(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.avg_pool(x)
        x = x.view(-1, 576)
        x = self.fc(x)
        return x
